# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397815128515                   -0.90    5.0   26.4ms
  2   -8.400245566346       -2.61       -1.74    1.0   18.8ms
  3   -8.400400252671       -3.81       -2.94    1.5   19.7ms
  4   -8.400427745268       -4.56       -2.91    3.2   46.8ms
  5   -8.400427973468       -6.64       -3.07    1.0   19.1ms
  6   -8.400428149737       -6.75       -5.10    1.0   19.1ms
  7   -8.400428155920       -8.21       -4.45    3.8   27.2ms
  8   -8.400428156273       -9.45       -5.43    2.0   22.3ms
  9   -8.400428156277      -11.46       -6.76    1.0   35.9ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397786154381                   -0.90           5.0    470ms
  2   -8.400384269380       -2.59       -1.79   0.80    2.0    510ms
  3   -8.400422739979       -4.41       -3.05   0.80    1.0    174ms
  4   -8.400428115929       -5.27       -3.43   0.80    2.8   64.2ms
  5   -8.400428154424       -7.41       -4.55   0.80    1.2   18.0ms
  6   -8.400428156252       -8.74       -5.68   0.80    2.0   19.4ms
  7   -8.400428156277      -10.62       -6.20   0.80    2.5   21.0ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +0.837666085393                   -1.07    2.79s
  2   -1.549850478857        0.38       -0.65    116ms
  3   -3.911243537882        0.37       -0.41   39.7ms
  4   -4.966350343338        0.02       -0.51   39.1ms
  5   -6.911417040925        0.29       -0.56   63.9ms
  6   -7.646878115449       -0.13       -0.95   39.6ms
  7   -7.987326929088       -0.47       -1.31   29.1ms
  8   -8.131779783726       -0.84       -1.54   29.3ms
  9   -8.187187524969       -1.26       -1.59   29.0ms
 10   -8.254598245266       -1.17       -1.42   53.3ms
 11   -8.295358108220       -1.39       -1.52   49.9ms
 12   -8.333326726225       -1.42       -1.81   39.3ms
 13   -8.362863160808       -1.53       -2.11   33.0ms
 14   -8.365116188135       -2.65       -2.85   29.7ms
 15   -8.378533438367       -1.87       -2.93   29.1ms
 16   -8.386865130848       -2.08       -2.59   32.2ms
 17   -8.39

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397804603578                   -0.90    5.2   36.0ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -8.400427979380                   -1.78    25.2s
  2   -8.400428156277       -6.75       -4.02    2.45s
  3   -8.400428156277      -14.75       -7.81    112ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.6664287130821306e-7
|ρ_newton - ρ_scfv| = 1.944982515687424e-7
|ρ_newton - ρ_dm|   = 4.6076855876029103e-7
